<a href="https://colab.research.google.com/github/MikeSalnikov/Recommend_systems/blob/main/hw_webinar_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание № 2

In [ ]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
# from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
# from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [ ]:
data = pd.read_csv('./retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [ ]:
# посчитаю сумму логарифмов: 

summary = np.log(data_train.groupby(['item_id'])['user_id'].count()).sum()

In [ ]:
# посчитаю веса: каждый логарифм поделю на сумму логарифмов: 

weights = np.log(data_train.groupby(['item_id'])['user_id'].count()) / summary

In [ ]:
# сумма весов должна равняться 1: 

weights.sum().round(5) == 1

True

In [ ]:
# создаю датафрейм с весами: 

items_weights = pd.DataFrame({'weight' : weights.array}, index=weights.index )

In [ ]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    recs = np.random.choice(items_weights.index, size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [ ]:
%%time

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

Wall time: 3.27 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[970708, 1014061, 847127, 5568894, 908015]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1049131, 6904759, 1036544, 1105623, 1128812]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [ ]:
result = pd.read_csv('./predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [ ]:
def bought_list_to_array(bought_list):
    p = bought_list.strip('[]\n').split(' ')
    p = list(filter(None, p))
    p = list(map(lambda x: int(x.strip()), p))
    return np.array(p)

def recommended_list_to_array(recommended_list):
    p = recommended_list.strip('[]').split(', ')
    p = list(map(int, p))
    return np.array(p)

def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = bought_list_to_array(bought_list)
    recommended_list = recommended_list_to_array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

Precision@5:

In [ ]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

0.0005876591576885406

In [ ]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.15523996082272082

In [ ]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.033594515181195064

In [ ]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual']), axis=1).mean()

0.03525954946131262

In [ ]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual']), axis=1).mean()

0.03614103819784544

In [ ]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual']), axis=1).mean()

0.1799869409076044

Алгоритм 'own_purchases' показывает лучшее качество.

Precision@3:

In [ ]:
result.apply(lambda row: precision_at_k(row['random_recommendation'], row['actual'], 3), axis=1).mean()

0.000652954619653934

In [ ]:
result.apply(lambda row: precision_at_k(row['popular_recommendation'], row['actual'], 3), axis=1).mean()

0.13777342474698054

In [ ]:
result.apply(lambda row: precision_at_k(row['itemitem'], row['actual'], 3), axis=1).mean()

0.033463924257264174

In [ ]:
result.apply(lambda row: precision_at_k(row['cosine'], row['actual'], 3), axis=1).mean()

0.03525954946131246

In [ ]:
result.apply(lambda row: precision_at_k(row['tfidf'], row['actual'], 3), axis=1).mean()

0.03819784524975512

In [ ]:
result.apply(lambda row: precision_at_k(row['own_purchases'], row['actual'], 3), axis=1).mean()

0.19833496571988093

Алгоритм 'own_purchases' показывает лучшее качество.

In [ ]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = bought_list_to_array(bought_list)
    recommended_list = recommended_list_to_array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

Recall@5:

In [ ]:
result.apply(lambda row: recall_at_k(row['random_recommendation'], row['actual']), axis=1).mean()

4.244797579826341e-05

In [ ]:
result.apply(lambda row: recall_at_k(row['popular_recommendation'], row['actual']), axis=1).mean()

0.024995563805434516

In [ ]:
result.apply(lambda row: recall_at_k(row['itemitem'], row['actual']), axis=1).mean()

0.005382921681330391

In [ ]:
result.apply(lambda row: recall_at_k(row['cosine'], row['actual']), axis=1).mean()

0.005180113732343748

In [ ]:
result.apply(lambda row: recall_at_k(row['tfidf'], row['actual']), axis=1).mean()

0.005172957067012944

In [ ]:
result.apply(lambda row: recall_at_k(row['own_purchases'], row['actual']), axis=1).mean()

0.02667473682464075

Алгоритм 'own_purchases' показывает лучшее качество.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
# your_code